In [1]:
import os
import ftplib
import pandas as pd
from glob import glob
import ahocorasick
from Bio import SeqIO
from tqdm import tqdm

In [2]:
filtered_peps = pd.read_table('/mnt/nfs/wangd/project/10154parquet/res/gca_peptides_for_deeplc_95thperc_observations_ms2pip_by_filtered.tsv')

In [3]:
filtered_peps

,sequence_x,protein_accessions,charge,scan_number,peptidoform,exp_mass_to_charge,calc_mass_to_charge,seq,tr,modifications,...,dot_product,count_B,count_Y,corrected_pearsonr_B,corrected_pearsonr_Y,corrected_dot_product,total_ions,number_peaks,signal_to_noise,diff_highest_lowest
0,AAAAAAAAAPAAAATAATTAATTAATAAQ,GCA_109298,2,28971,AAAAAAAAAPAAAATAATTAATTAATAAQ,1172.107178,1171.600967,AAAAAAAAAPAAAATAATTAATTAATAAQ,4215.9185,NaN,...,0.368816,28,28,0.938671,0.916177,1.854847,56,201,8.335360,382.334717
1,AAAAPAPPAGPR,GCA_161748,2,9337,AAAAPAPPAGPR,524.284728,523.790718,AAAAPAPPAGPR,1338.6000,NaN,...,0.054925,11,11,-0.051957,1.276448,1.224491,22,91,4.659606,293.245368
2,AAAAPEASPSPGGDAAWSEAGPGPR,GCA_125965,2,19121,AAAAPEASPSPGGDAAWSEAGPGPR,1139.535889,1139.030177,AAAAPEASPSPGGDAAWSEAGPGPR,2500.9456,NaN,...,0.804288,24,24,0.914510,1.865369,2.779879,48,134,9.519287,623.096960
3,AAAFYKNILGAQVSEVVPLPEHGVSVVFVNLGNTK,GCA_133924,4,62173,AAAFYKNILGAQ[Deamidated]VSEVVPLPEHGVSVVFVNLGNTK,918.255432,917.997627,AAAFYKNILGAQVSEVVPLPEHGVSVVFVNLGNTK,5808.4260,12|Deamidated,...,0.598592,34,34,0.731502,2.136918,2.868420,68,233,6.354093,332.768632
4,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,GCA_60405,4,28430,AAAGDLGGDHLAFSC[Carbamidomethyl]DVAKEHDVQNTFEEMEK,873.644649,873.642418,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,3506.6000,15|Carbamidomethyl,...,0.234048,31,31,0.274421,2.112054,2.386475,62,294,7.324008,449.853470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,4,34975,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVN[Dea...,967.694946,967.427389,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,4089.2166,3|Carbamidomethyl|15|Carbamidomethyl|19|Carbam...,...,0.215309,34,34,0.258653,2.173789,2.432442,68,267,11.936250,881.325484
8549,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,4,35579,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVNAC[C...,967.692444,967.427389,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,3992.6436,3|Carbamidomethyl|15|Carbamidomethyl|19|Carbam...,...,0.565984,34,34,0.716063,2.064077,2.780139,68,304,13.346966,592.653132
8550,YWMHFCGGSLIHPQWVLTAAHCLGPDVK,"GCA_284206,GCA_284208",5,36949,YWM[Oxidation]HFC[Carbamidomethyl]GGSLIHPQ[Dea...,660.522216,660.314440,YWMHFCGGSLIHPQWVLTAAHCLGPDVK,4513.1000,3|Oxidation|6|Carbamidomethyl|22|Carbamidometh...,...,0.509793,27,27,0.716000,1.624169,2.340169,54,94,4.179343,195.553136
8551,YYNAKTEDKVR,GCA_154212,3,5165,YYN[Deamidated]AKTEDKVR,463.234832,463.233260,YYNAKTEDKVR,1137.2000,3|Deamidated,...,0.790630,10,10,0.586982,1.499682,2.086664,20,151,8.627556,441.274258


In [4]:
proteins_accessions = filtered_peps["protein_accessions"].to_list()
db_prots_ids = set()
for ids in proteins_accessions:
    for id in ids.split(","):
        db_prots_ids.add(id)

In [5]:
db_prots = dict()
parsed_file = SeqIO.parse("/mnt/nfs/wangd/project/paper_db/GRCh38r110_GCA97s_coding_proteins_19Jul23-decoy.fa", 'fasta')
for record in parsed_file:
    if record.id in db_prots_ids:
        db_prots[record.id] = str(record.seq)

In [16]:
len(db_prots)

4191

In [6]:
def get_protein_seq(protein_accessions):
    seqs = []
    proteins = protein_accessions.split(",")
    for protein in proteins:
        seq = db_prots[protein]
        seqs.append(seq)
    return seqs

In [7]:
filtered_peps["protein_seq"] = filtered_peps["protein_accessions"].apply(get_protein_seq)

In [8]:
filtered_peps

,sequence_x,protein_accessions,charge,scan_number,peptidoform,exp_mass_to_charge,calc_mass_to_charge,seq,tr,modifications,...,count_B,count_Y,corrected_pearsonr_B,corrected_pearsonr_Y,corrected_dot_product,total_ions,number_peaks,signal_to_noise,diff_highest_lowest,protein_seq
0,AAAAAAAAAPAAAATAATTAATTAATAAQ,GCA_109298,2,28971,AAAAAAAAAPAAAATAATTAATTAATAAQ,1172.107178,1171.600967,AAAAAAAAAPAAAATAATTAATTAATAAQ,4215.9185,NaN,...,28,28,0.938671,0.916177,1.854847,56,201,8.335360,382.334717,[MVLLESEQFLTELTRLFQKCRTSGSVYITLKKYDGRTKPIPKKGT...
1,AAAAPAPPAGPR,GCA_161748,2,9337,AAAAPAPPAGPR,524.284728,523.790718,AAAAPAPPAGPR,1338.6000,NaN,...,11,11,-0.051957,1.276448,1.224491,22,91,4.659606,293.245368,[MEPAAGFLSPRPFQRAAAAPAPPAGPRPPPSALRGPELEMLAGLP...
2,AAAAPEASPSPGGDAAWSEAGPGPR,GCA_125965,2,19121,AAAAPEASPSPGGDAAWSEAGPGPR,1139.535889,1139.030177,AAAAPEASPSPGGDAAWSEAGPGPR,2500.9456,NaN,...,24,24,0.914510,1.865369,2.779879,48,134,9.519287,623.096960,[MSRQSTLYSFFPKSPALSDANKASARASREGGRAAAAPEASPSPG...
3,AAAFYKNILGAQVSEVVPLPEHGVSVVFVNLGNTK,GCA_133924,4,62173,AAAFYKNILGAQ[Deamidated]VSEVVPLPEHGVSVVFVNLGNTK,918.255432,917.997627,AAAFYKNILGAQVSEVVPLPEHGVSVVFVNLGNTK,5808.4260,12|Deamidated,...,34,34,0.731502,2.136918,2.868420,68,233,6.354093,332.768632,[MARVLKAAAANAVGLFSRLQAPIPTVRASSTSQPLDQVTGSVWNL...
4,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,GCA_60405,4,28430,AAAGDLGGDHLAFSC[Carbamidomethyl]DVAKEHDVQNTFEEMEK,873.644649,873.642418,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,3506.6000,15|Carbamidomethyl,...,31,31,0.274421,2.112054,2.386475,62,294,7.324008,449.853470,[MDKVCAVFGGSRGIGRAVAQLMARKGYRLAVIARNLEGAKAAAGD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,4,34975,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVN[Dea...,967.694946,967.427389,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,4089.2166,3|Carbamidomethyl|15|Carbamidomethyl|19|Carbam...,...,34,34,0.258653,2.173789,2.432442,68,267,11.936250,881.325484,[MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLAN...
8549,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,4,35579,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVNAC[C...,967.692444,967.427389,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,3992.6436,3|Carbamidomethyl|15|Carbamidomethyl|19|Carbam...,...,34,34,0.716063,2.064077,2.780139,68,304,13.346966,592.653132,[MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLAN...
8550,YWMHFCGGSLIHPQWVLTAAHCLGPDVK,"GCA_284206,GCA_284208",5,36949,YWM[Oxidation]HFC[Carbamidomethyl]GGSLIHPQ[Dea...,660.522216,660.314440,YWMHFCGGSLIHPQWVLTAAHCLGPDVK,4513.1000,3|Oxidation|6|Carbamidomethyl|22|Carbamidometh...,...,27,27,0.716000,1.624169,2.340169,54,94,4.179343,195.553136,[MLSLLLLALPVLASPAYAAPAPVQALQQAGIVGGQEAPRSKWPWQ...
8551,YYNAKTEDKVR,GCA_154212,3,5165,YYN[Deamidated]AKTEDKVR,463.234832,463.233260,YYNAKTEDKVR,1137.2000,3|Deamidated,...,10,10,0.586982,1.499682,2.086664,20,151,8.627556,441.274258,[MAEGNHRKKPLKVLESLGKDFLTGVLDNLVEQNVLNWKEEEKKKY...


In [9]:
auto = ahocorasick.Automaton()

for seq in db_prots.values():
    auto.add_word(seq, seq)

auto.make_automaton()

In [10]:
# grch_prots = dict()
# grch_file = SeqIO.parse("/mnt/nfs/wangd/project/paper_db/pep_files/save_tmp/Homo_sapiens-GRCh38_all-pep.fa", 'fasta')
# for record in grch_file: 
#     try:
#         grch_prots[str(record.seq)].append(record.description)
#     except KeyError:
#         grch_prots[str(record.seq)] = [record.description]

In [11]:
peps_prots = {}
for prot_file in tqdm(glob('/mnt/nfs/wangd/project/paper_db/pep_files/*.fa')):
    prots_dict = {}
    parsed_file = SeqIO.parse(prot_file, 'fasta')
    sample_name = prot_file.split('/')[-1].split('-')[1]

    for record in parsed_file:
        try:
            prots_dict[str(record.seq)].append(sample_name + ' ' +  str(record.id) + ' ' + str(record.description))
        except KeyError:
            prots_dict[str(record.seq)] = [sample_name + ' ' + str(record.id) + ' ' + str(record.description)]

    found_prots = set()
    
    for prot_seq in prots_dict.keys():
        for end_ind, found in auto.iter(prot_seq):
            found_prots.add(found)
            try:
                peps_prots[found].append(prots_dict[prot_seq])
            except KeyError:
                peps_prots[found] = [prots_dict[prot_seq]]

100%|██████████| 97/97 [02:35<00:00,  1.60s/it]


In [97]:
samples = set()
proteins = set()
genes = set()
transcripts = set()
gene_biotypes = set()
gene_symbols = set()
def get_info(seq):
    all_res = []
    for infos in peps_prots[seq]:
        for info in infos:
            info_split = info.split(" ")
            sample = info_split[0]
            protein = info_split[1]
            gene = [x for x in info_split if x.startswith('gene:')][0].split(':')[-1]
            transcript = [x for x in info_split if x.startswith('transcript:')][0].split(':')[-1]
            gene_biotype = [x for x in info_split if x.startswith('gene_biotype:')][0].split(':')[-1]
            try:
                gene_symbol = [x for x in info_split if x.startswith('gene_symbol:')][0].split(':')[-1]
            except IndexError:
                gene_symbol = ""
            
            res = [sample, protein, gene, transcript, gene_biotype, gene_symbol]
            all_res.append(res)
            
            samples.add(sample)
            proteins.add(protein)
            genes.add(gene)
            transcripts.add(transcript)
            gene_biotypes.add(gene_biotype)
            gene_symbols.add(gene_symbol)
            
    return all_res

In [98]:
def get_protein_info(protein_seqs):
    all_seqs_infos = []
    for seq in protein_seqs:
        all_seqs_infos.append(get_info(seq))
    return all_seqs_infos

In [99]:
filtered_peps["protein_infos"] = filtered_peps["protein_seq"].apply(get_protein_info)

In [100]:
print("Number of samples: {}".format(len(samples)))
print("Number of proteins: {}".format(len(proteins)))
print("Number of genes: {}".format(len(genes)))
print("Number of transcripts: {}".format(len(transcripts)))
print("Number of gene_biotypes: {}".format(len(gene_biotypes)))
print("Number of gene_symbols: {}".format(len(gene_symbols)))

Number of samples: 97
Number of proteins: 63836
Number of genes: 45039
Number of transcripts: 63839
Number of gene_biotypes: 4
Number of gene_symbols: 2514


In [101]:
filtered_peps

,sequence_x,protein_accessions,charge,scan_number,peptidoform,exp_mass_to_charge,calc_mass_to_charge,seq,tr,modifications,...,count_Y,corrected_pearsonr_B,corrected_pearsonr_Y,corrected_dot_product,total_ions,number_peaks,signal_to_noise,diff_highest_lowest,protein_seq,protein_infos
0,AAAAAAAAAPAAAATAATTAATTAATAAQ,GCA_109298,2,28971,AAAAAAAAAPAAAATAATTAATTAATAAQ,1172.107178,1171.600967,AAAAAAAAAPAAAATAATTAATTAATAAQ,4215.9185,NaN,...,28,0.938671,0.916177,1.854847,56,201,8.335360,382.334717,[MVLLESEQFLTELTRLFQKCRTSGSVYITLKKYDGRTKPIPKKGT...,"[[[GCA_009914755.4, ENSP05220080001.1, ENSG052..."
1,AAAAPAPPAGPR,GCA_161748,2,9337,AAAAPAPPAGPR,524.284728,523.790718,AAAAPAPPAGPR,1338.6000,NaN,...,11,-0.051957,1.276448,1.224491,22,91,4.659606,293.245368,[MEPAAGFLSPRPFQRAAAAPAPPAGPRPPPSALRGPELEMLAGLP...,"[[[GCA_018505865.1, ENSP05245091521.1, ENSG052..."
2,AAAAPEASPSPGGDAAWSEAGPGPR,GCA_125965,2,19121,AAAAPEASPSPGGDAAWSEAGPGPR,1139.535889,1139.030177,AAAAPEASPSPGGDAAWSEAGPGPR,2500.9456,NaN,...,24,0.914510,1.865369,2.779879,48,134,9.519287,623.096960,[MSRQSTLYSFFPKSPALSDANKASARASREGGRAAAAPEASPSPG...,"[[[GCA_018467005.1, ENSP04855059600.1, ENSG048..."
3,AAAFYKNILGAQVSEVVPLPEHGVSVVFVNLGNTK,GCA_133924,4,62173,AAAFYKNILGAQ[Deamidated]VSEVVPLPEHGVSVVFVNLGNTK,918.255432,917.997627,AAAFYKNILGAQVSEVVPLPEHGVSVVFVNLGNTK,5808.4260,12|Deamidated,...,34,0.731502,2.136918,2.868420,68,233,6.354093,332.768632,[MARVLKAAAANAVGLFSRLQAPIPTVRASSTSQPLDQVTGSVWNL...,"[[[GCA_018472705.1, ENSP05065012782.1, ENSG050..."
4,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,GCA_60405,4,28430,AAAGDLGGDHLAFSC[Carbamidomethyl]DVAKEHDVQNTFEEMEK,873.644649,873.642418,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,3506.6000,15|Carbamidomethyl,...,31,0.274421,2.112054,2.386475,62,294,7.324008,449.853470,[MDKVCAVFGGSRGIGRAVAQLMARKGYRLAVIARNLEGAKAAAGD...,"[[[GCA_018466845.1, ENSP04840088211.1, ENSG048..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,4,34975,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVN[Dea...,967.694946,967.427389,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,4089.2166,3|Carbamidomethyl|15|Carbamidomethyl|19|Carbam...,...,34,0.258653,2.173789,2.432442,68,267,11.936250,881.325484,[MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLAN...,"[[[GCA_018504645.1, ENSP05210059631.1, ENSG052..."
8549,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,4,35579,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVNAC[C...,967.692444,967.427389,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,3992.6436,3|Carbamidomethyl|15|Carbamidomethyl|19|Carbam...,...,34,0.716063,2.064077,2.780139,68,304,13.346966,592.653132,[MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLAN...,"[[[GCA_018504645.1, ENSP05210059631.1, ENSG052..."
8550,YWMHFCGGSLIHPQWVLTAAHCLGPDVK,"GCA_284206,GCA_284208",5,36949,YWM[Oxidation]HFC[Carbamidomethyl]GGSLIHPQ[Dea...,660.522216,660.314440,YWMHFCGGSLIHPQWVLTAAHCLGPDVK,4513.1000,3|Oxidation|6|Carbamidomethyl|22|Carbamidometh...,...,27,0.716000,1.624169,2.340169,54,94,4.179343,195.553136,[MLSLLLLALPVLASPAYAAPAPVQALQQAGIVGGQEAPRSKWPWQ...,"[[[GCA_018471095.1, ENSP04990077550.1, ENSG049..."
8551,YYNAKTEDKVR,GCA_154212,3,5165,YYN[Deamidated]AKTEDKVR,463.234832,463.233260,YYNAKTEDKVR,1137.2000,3|Deamidated,...,10,0.586982,1.499682,2.086664,20,151,8.627556,441.274258,[MAEGNHRKKPLKVLESLGKDFLTGVLDNLVEQNVLNWKEEEKKKY...,"[[[GCA_018466855.1, ENSP04845078198.1, ENSG048..."
